In [15]:
import os, urllib.request, json, subprocess, geopandas, pandas, psycopg2
from datetime import datetime

In [16]:
os.environ['SHAPE_ENCODING'] = "utf-8"
basePath = 'C:\\saprog\\projeto'
outputPath = 'C:\\saprog\\projeto\\output'
os.chdir(basePath)

def deleteAndCreateOutput():
    print ('A limpar e criar directoria de outputs...')
    outDir = os.path.join(basePath, 'output')
    if os.path.isdir(outDir):
        shutil.rmtree(outDir)
    os.mkdir(outDir)

deleteAndCreateOutput()

A limpar e criar directoria de outputs...


In [17]:
print('A ler shapefile...')
caop = geopandas.read_file('caop.shp',encoding='utf-8')
# caop.plot()
# caop.head()
# gdf = caop.loc[caop['Concelho'] == 'Bombarral']
# gdf.head()

A ler shapefile...


In [18]:
print('A executar Dissolve...')
distritos = caop.dissolve(by = 'Distrito')
# distritos.plot()
# distritos.head(1)

A executar Dissolve...


In [19]:
print('A extrair centroides das regiões admninistrativas...')
shpOutput = os.path.join(outputPath, 'centroides.shp')

# dataProj = distritos.to_crs(epsg = 3763)
# centroides = dataProj.centroid
centroides = distritos.centroid
centroides.to_file(shpOutput, encoding='utf-8')
# centroides.plot()
# centroides.head()

A extrair centroides das regiões admninistrativas...
<ipython-input-19-5a8a41490fe0>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroides = distritos.centroid


In [26]:
coordx = centroides.x.to_dict()
coordy = centroides.y.to_dict()
# print(coordx)
# print(coordy)

def getCoordinates(dicX, dicY):
    '''
    Extração de coordenadas geográficas úteis ao harvest de dados meteorológicos.
    Devolve um dicionário onde as chaves são os distritos, à qual está associado um tuplo com as coordenadas Lat Long.
    '''
    coord = [dicX, dicY]
    coordDic = {}
    for i in dicX.keys():
        coordDic[i] = tuple(coordDic[i] for coordDic in coord)
    return coordDic

print('A extrair coodenadas geográficas...')
coord = getCoordinates(coordx, coordy)
print(coord)

A extrair coodenadas geográficas...
{'Aveiro': (-8.46838037713987, 40.7236925098494), 'Beja': (-7.94388147040214, 37.82970078780623), 'Braga': (-8.309815844734775, 41.55294943650094), 'Bragança': (-6.859336412433179, 41.509404985269676), 'Castelo Branco': (-7.5016170094379, 39.946483706164535), 'Coimbra': (-8.33590045610553, 40.204414004264905), 'Faro': (-8.13168379460421, 37.24368427672163), 'Guarda': (-7.229585420073208, 40.641320980615), 'Leiria': (-8.775241866061927, 39.717272858669155), 'Lisboa': (-9.163514891800212, 38.9967494344431), 'Portalegre': (-7.620439565448195, 39.190063045702445), 'Porto': (-8.352739071777673, 41.22479245090154), 'Santarém': (-8.477568721428257, 39.293494949281204), 'Setúbal': (-8.652857331305352, 38.318451316051366), 'Viana do Castelo': (-8.50709117586541, 41.87785654627992), 'Vila Real': (-7.631700989514108, 41.55496982213632), 'Viseu': (-7.870908401501086, 40.79887555573913), 'Évora': (-7.8417966523604, 38.603868659253635)}


In [21]:
# Carregamento de shapefile na bd
# command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
#           "-f", "PostgreSQL",
#           "PG:host=localhost user=postgres dbname=meteo password=3763",
#           shpOutput,
#           "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
#           " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "centroides", "-overwrite"]
# subprocess.check_call(command)

In [22]:
apiKey = '44cfad7f82ec61d3f420de3201b703d4'

def getWeather(coordDic, apikey):
    '''
    Função para recolha dos dados meteorológicos provenientes da OpenWeather.
    Devolve uma dataframe das váriáveis meteorológicas por distrito.
    '''
    forecast = []
    for item in coord.items():
        lat = str(item[1][1])
        long = str(item[1][0])
        url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + long + \
            '&exclude=minutely,hourly,daily,alerts&appid=' + apiKey + '&units=metric'
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            districtForecast = {}
            districtForecast['distrito'] = item[0]
            districtForecast['forecast_date'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%d-%m-%Y')
            districtForecast['forecast_time'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%H:%M:%S')
            main = data.get('current').get('weather')
            for item in main:
                districtForecast['weather_desc'] = item.get('main')
            districtForecast['temperature'] = data.get('current').get('temp')
            districtForecast['feels_like'] = data.get('current').get('feels_like')
            districtForecast['pressure'] = data.get('current').get('pressure')
            districtForecast['humidity'] = data.get('current').get('humidity')
            districtForecast['dew_point'] = data.get('current').get('dew_point')
            districtForecast['ultrav_index'] = data.get('current').get('uvi')
            districtForecast['wind_speed'] = data.get('current').get('wind_speed')
            districtForecast['wind_deg'] = data.get('current').get('wind_deg')
            forecast.append(districtForecast)
    forecast_df = pandas.DataFrame(getWeather(coord, apiKey))
    return forecast


forecast_df.head(-1)

,distrito,forecast_date,forecast_time,weather_desc,temperature,feels_like,pressure,humidity,dew_point,ultrav_index,wind_speed,wind_deg
0,Aveiro,14-03-2021,19:10:56,Clear,13.17,8.16,1026,88,11.23,0,7.72,350
1,Beja,14-03-2021,19:10:56,Clear,18.00,14.17,1023,45,5.90,0,4.12,30
2,Braga,14-03-2021,19:10:56,Clear,12.64,8.27,1026,82,9.65,0,6.17,340
3,Bragança,14-03-2021,19:10:57,Clear,9.27,7.53,1026,68,3.68,0,0.52,3
4,Castelo Branco,14-03-2021,19:10:57,Clouds,8.33,4.99,1023,53,-0.60,0,1.79,193
5,Coimbra,14-03-2021,19:10:57,Clear,12.97,9.91,1023,89,11.20,0,4.92,7
6,Faro,14-03-2021,19:10:58,Clear,16.00,13.43,1022,72,10.96,0,4.12,270
7,Guarda,14-03-2021,19:10:58,Clear,8.86,5.59,1023,53,-0.17,0,1.79,193
8,Leiria,14-03-2021,19:10:58,Clear,12.85,9.32,1026,76,8.73,0,4.63,360
9,Lisboa,14-03-2021,19:10:58,Clear,13.53,12.94,1025,79,9.96,0,0.89,203


In [25]:
password = open(os.path.join('pw.txt'), 'r').readline()
password = str(password)

con = psycopg2.connect(dbname='meteo', user='postgres', password=password, host='localhost', port='5432')

def dfToPgSQL(conn, df, table):
    '''
    Utilização da função psycopg2.extras.execute_values()
    para inserção da data frame colhida na tabela PostGreSQL "forecast"
    '''
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        psycopg2.extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("dfToPgSQL() done")
    cursor.close()

dfToPgSQL(con, forecast_df, 'forecast')

dfToPgSQL() done
